In [1]:
!pip install torchaudio

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
!pip install pydub
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
import os
import pandas as pd
import sys
sys.path.insert(0, os.path.abspath('../lib'))
#import config
import config_DK_AST
#import config

from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import datetime
from torchvision import datasets
import librosa
import librosa.display
import os
import skimage.util
import pickle
import math
import collections
import pandas as pd
from sklearn.utils import shuffle
from scipy import signal
import scipy
import matplotlib.pyplot as plt
from scipy.signal import butter , filtfilt
import scipy.io as sio
import scipy.io.wavfile
import copy
from PyTorch import config_pytorch
from PyTorch.runTorch_AST_DK import ASTModel
from evaluate import get_results
from pydub import AudioSegment
import math
import pathlib
import random
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import DataLoader, Dataset, random_split
import time
from sklearn.metrics import accuracy_score




/opt/conda/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Few Global params

In [4]:
data_dir = os.path.join(os.path.pardir, 'data', 'audio')
plot_dir = os.path.join(os.path.pardir, 'outputs',  'plots')
model_dir = os.path.join(os.path.pardir, 'outputs', 'models') # Model sub-directory created in config_keras or config_pytorch
# this is the batch_size of the dataloader
loader_bat_size = 4

# set this to True if you want to call the bandpass filter
call_filter = False
# This is the duration of the max. len train file. Due to randomization the length should decrease.
max_train_mel_len = None

## Class Definitions for Transformations

In [5]:
#https://jonathanbgn.com/2021/08/30/audio-augmentation.html
class RandomBackgroundNoise:
    def __init__(self,  noise_dir ,sample_rate=config_DK_AST.rate, min_snr_db=0, max_snr_db=10):
        self.sample_rate = sample_rate
        self.min_snr_db = min_snr_db
        self.max_snr_db = max_snr_db

        if not os.path.exists(noise_dir):
            raise IOError(f'Noise directory `{noise_dir}` does not exist')
        # find all WAV files including in sub-folders:
        self.noise_files_list = list(pathlib.Path(noise_dir).glob('**/*.wav'))
        if len(self.noise_files_list) == 0:
            raise IOError(f'No .wav file found in the noise directory `{noise_dir}`')

    def __call__(self, audio_data):
        random_noise_file = random.choice(self.noise_files_list)
        effects = [
            ['remix', '1'], # convert to mono
            ['rate', str(self.sample_rate)], # resample
        ]
        noise, _ = torchaudio.sox_effects.apply_effects_file(random_noise_file, effects, normalize=True)
        audio_length = audio_data.shape[-1]
        noise_length = noise.shape[-1]
        if noise_length > audio_length:
            offset = random.randint(0, noise_length-audio_length)
            noise = noise[..., offset:offset+audio_length]
        elif noise_length < audio_length:
            noise = torch.cat([noise, torch.zeros((noise.shape[0], audio_length-noise_length))], dim=-1)

        snr_db = random.randint(self.min_snr_db, self.max_snr_db)
        snr = math.exp(snr_db / 10)
        audio_power = audio_data.norm(p=2)
        noise_power = noise.norm(p=2)
        scale = snr * noise_power / audio_power

        return (scale * audio_data + noise ) / 2
 


In [6]:
class RandomSpeedChange:
    def __init__(self, sample_rate = config_DK_AST.rate):
        self.sample_rate = sample_rate

    def __call__(self, audio_data):
        speed_factor = random.choice([0.9, 1.0, 1.1])
        if speed_factor == 1.0: # no change
            return audio_data

        # change speed and resample to original rate:
        sox_effects = [
            ["speed", str(speed_factor)],
            ["rate", str(self.sample_rate)],
        ]
        transformed_audio, _ = torchaudio.sox_effects.apply_effects_tensor(
            audio_data, self.sample_rate, sox_effects)
        return transformed_audio
 






In [7]:
class ComposeTransform:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, audio_data):
        for t in self.transforms:
            audio_data = t(audio_data)
        return audio_data

# 2 kind of transformations on the incoming training data
compose_transform_aug_noise = ComposeTransform([
    #RandomClip(sample_rate=sample_rate, sequence_length=64000),
    RandomSpeedChange(config_DK_AST.rate  ),
    RandomBackgroundNoise(noise_dir = "../data/audio/noise" )])

compose_transform_aug_signal = ComposeTransform([
    #RandomClip(sample_rate=sample_rate, sequence_length=64000),
    RandomSpeedChange(config_DK_AST.rate  ),
    RandomBackgroundNoise(noise_dir = "../data/audio/signal" , min_snr_db=5, max_snr_db=20)])




In [8]:
class AudioUtil(ComposeTransform,RandomSpeedChange,RandomBackgroundNoise):
## Function to apply Spectograms    
    @staticmethod
    def spec_augment(feat, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        print("......APplying spec augment.....")
        _, n_mels, n_steps = feat.shape
        #print("n_mels = " + str(n_mels))
        #print("n_steps = " + str(n_steps))
        mask_value = feat.mean()
        #print("mask_val = " + str(mask_value))
        feat_aug = feat
        freq_mask_param = max_mask_pct * n_mels
        
        for _ in range(n_freq_masks):
            feat_aug = T.FrequencyMasking(freq_mask_param)(feat_aug, mask_value)
            
        
        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            feat_aug = T.TimeMasking(time_mask_param)(feat_aug, mask_value)
            
        return feat_aug
    
    # This function gets the expected number of spectograms for a file.
    # THis is needed because randomization/length and padding at times add an extra spectogram and we need to compare this expected number with the 
    # one that is returned from reshape
    
    ## Function that takes a file names and returns a list of spectograms
    @staticmethod
    def get_train_feat_list(filename_str,label, min_duration = config_DK_AST.min_duration):
        
        filename = os.path.join(data_dir, filename_str)
        print("inside get feat list , processing ->" + str(filename))
        label_duration = librosa.get_duration(filename = filename)
        if label_duration >= min_duration:
            #print("filename = " + str(filename))
            #call the function to process the sound file
            feat_aug, y_temp , bugs = AudioUtil.process_wav (filename,label_duration,label )
            if (y_temp != "FAIL" ):
                #print("File processeing successful . SHape being returned is   " + str(feat_aug.shape))
                final_feat_list = reshape_feat_pt_no_pad(feat_aug,label)
                #print("final_feat_list length = " + str(len(final_feat_list)))
                return (final_feat_list,"success")
                    
                           
            #when process_wav has processing issues    
            else:
                print("a failure has occured during processing, return a tensor of zeros")
                feat_aug = torch.zeros(1, config_DK_AST.win_size,config_DK_AST.n_feat)
                #print("Shape being returned from process_wav = " + str(feat_aug.shape))
                #final_feat_list = reshape_feat_pt_no_pad(feat_aug,max_length = max_length)
                #print("final_feat_list length , returning from reshape = " + str(len(final_feat_list)))
                final_feat_list = list(feat_aug)
                return final_feat_list,999
                
                
        #when there is padding needed and         
        else:
            print("inside shorter files")
            size_to_pad = min_duration - label_duration 
            #print("size_to_pad = " + str(size_to_pad))
            #calling process_wav first to ensure that the 'signal' is centered
            feat, y_temp , bugs = AudioUtil.process_wav (filename,label_duration,label,pad = True,filter_signal = call_filter )
            #print("shape of returned file from process_wav -> " + str(feat.shape))
            # adding tolerance because the randomization from augmentation sometimes reduces the file size < window size.
            # This causes downstream problem in reshape
            size = config_DK_AST.rate*(size_to_pad + .25*label_duration) + feat.shape[1]
            #print("size with which the file will be appende -> " + str(size))
            #print("about to apply padding")
            feat_padded = librosa.util.pad_center(feat, size=size , axis = 1,mode = 'mean')
            #print("shape of padded file = " + str(feat_padded.shape) + "and its type = " + str(type(feat_padded)))
            #convert to mel scale
            mel_spectrogram = T.MelSpectrogram(sample_rate=16000, n_fft=config_DK_AST.NFFT,win_length=config_DK_AST.win_size, 
                                                       hop_length=config_DK_AST.n_hop, center=True,
                                                       pad_mode="reflect", power=2.0, norm='slaney', onesided=True, n_mels=config_DK_AST.n_feat,
                                                       mel_scale="htk")
            
            melspec_temp = mel_spectrogram(torch.tensor(feat_padded))
            #print("After melspectrogram . Shape of feat ->" + str(melspec_temp.shape))
            try:
                db_obj= torchaudio.transforms.AmplitudeToDB(stype= 'power')
                feat = db_obj(melspec_temp)
                #print("after converting to mel scale , shape of feat = " + str(feat.shape))
                #print(" feat[2] =" + str(feat.shape[2]))
                    
                if feat.shape[2] < config_DK_AST.win_size :
                    #print("After power_to_db . Shape of feat ->" + str(feat.shape))
                    raise ValueError
                        
                else:
                    feat_aug = AudioUtil.spec_augment(feat, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1)
                    #print("POst spec_augment ...for filename = " + str(filename) + "shape of feat_aug = " + str(feat_aug.shape))
                    #librosa.display.specshow(feat)
                    #when the file is shorter than min duration then we'd return only 1 spectogram
                    final_feat_list = reshape_feat_pt_no_pad(feat_aug,label)
                    return final_feat_list,"success"
                    
            except ValueError:
                print("!!!!ValueError!!!!!!for filename " + str(filename) + "FILE SIZE < Win size ")
                final_feat_list = torch.zeros(1, config_DK_AST.win_size,config_DK_AST.n_feat)
                #print("final_feat_list length , returning from reshape = " + str(len(final_feat_list)))
                final_feat_list = list(final_feat_list)
                return final_feat_list,999
    
    ### Function to get the list of test features
    @staticmethod
    def get_test_fest_list(filename_str,label,min_duration = config_DK_AST.min_duration):
        filename = os.path.join(data_dir, filename_str)
        print("....inside get_test_fest_list....")
        label_duration = librosa.get_duration(filename = filename)
        if label_duration > min_duration:
            #print("AUdio duration of File " + str(filename) + "is more than min duration")
            signal_8k, rate = torchaudio.load(filename , normalize = True)
            #print("Signal 8K shape = " + str(signal_8k.shape))
            # resampling to 16 K as needed by AST
            transform = T.Resample(orig_freq = 8000,new_freq = 16000)
            signal_file = transform(signal_8k)
            #print("Signal 16K shape = " + str(signal_file.shape))
            #converting to mel scale
            mel_spectrogram = T.MelSpectrogram(sample_rate=16000, n_fft=config_DK_AST.NFFT,win_length=config_DK_AST.win_size, 
                                                   hop_length=config_DK_AST.n_hop, center=True,
                                                   pad_mode="reflect", power=2.0, norm='slaney', onesided=True, n_mels=config_DK_AST.n_feat,
                                                   mel_scale="htk")
            melspec = mel_spectrogram(signal_file)
            db_obj= torchaudio.transforms.AmplitudeToDB(stype= 'power')
            feat = db_obj(melspec)
            #print("After melspectrogram . Shape of feat ->" + str(melspec.shape))
            #feat = librosa.power_to_db(feat, ref=np.max)
            #print("After power_to_db . Shape of feat ->" + str(feat.shape))
                            
            # split into winsize x labels
            if feat.shape[2] < config_DK_AST.win_size:
                print("File->" + str(filename) + "is smaller than win size. Returning a 999 label")
                temp_feat = torch.zeros(1, config_DK_AST.win_size,config_DK_AST.n_feat)
                test_feat_list = list(temp_feat)
                return test_feat_list,999
                
            else:
                test_feat_list = reshape_feat_pt_no_pad(feat, label,size = config_DK_AST.win_size, step = config_DK_AST.step_size )
                #print("length of list from reshape = " + str(len(test_feat_list)))
                print("file " + str(filename) + "will have " + str(len(test_feat_list)) + " specgrams")
                return test_feat_list,"success"
                        
                
        else:            
            #print("Audio duration of File " + str(filename) + "is less than min duration")
            signal_8k, rate = torchaudio.load(filename , normalize = True)
            # resampling to 16 K as needed by AST
            transform = T.Resample(orig_freq = 8000,new_freq = 16000)
            signal_file = transform(signal_8k)
            #print("shape of signal file = " + str(signal_file.shape))
            num_rows, sig_len = signal_file.shape
            # the code below is to pad with 0
            len_begin = int(round(min_duration*1000 - label_duration*1000 ))
            pad_begin_len = random.randint(0,len_begin)
            #print("pad_begin_len = " + str(pad_begin_len))
            pad_end_len = (min_duration*1000 - label_duration*1000 - pad_begin_len)*(config_DK_AST.sr/1000)
            #print("pad_end_len = " + str(pad_end_len))
            # Pad with 0s
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            #print("pad_begin = " +str(pad_begin))
            pad_end = torch.zeros((num_rows, int(pad_end_len)))
            #print("pad_end = " +str(pad_end))
            signal_appended = torch.cat((pad_begin, signal_file, pad_end), 1)
            #print("signal_appended shape = " + str(signal_appended.shape))
            
            # now covert to mel scale
            mel_spectrogram = T.MelSpectrogram(sample_rate=16000, n_fft=config_DK_AST.NFFT,win_length=config_DK_AST.win_size, 
                                                   hop_length=config_DK_AST.n_hop, center=True,
                                                   pad_mode="reflect", power=2.0, norm='slaney', onesided=True, n_mels=config_DK_AST.n_feat,
                                                   mel_scale="htk")
            melspec = mel_spectrogram(signal_appended)
            db_obj= torchaudio.transforms.AmplitudeToDB(stype= 'power')
            feat = db_obj(melspec)
            
            try:
                if feat.shape[2] < config_DK_AST.win_size:
                    #print("After melspectrogram . Shape of feat ->" + str(melspec.shape))
                    #print("File->" + str(filename) + "being skipped because its < config_size")
                    #print return with a 999 label indicating that this is an invalid input
                    raise ValueError                 
                else:
                    #print("After power_to_db . Shape of feat ->" + str(feat.shape))
                    test_feat_list = reshape_feat_pt_no_pad(feat, label,size = config_DK_AST.win_size, step = config_DK_AST.step_size )
                    #print("file " + str(filename) + "will have " + str(len(test_feat_list)) + " specgrams")
                    test_feat_list = test_feat_list[0:1]
                    return test_feat_list,"success"
                    
                    
            except ValueError:
                print(" ValueError .for filename " + str(filename) + "FILE SIZE < Win size. Returning 999 ")
                test_feat_list = torch.zeros(1, config_DK_AST.win_size,config_DK_AST.n_feat)
                #test_feat_list = reshape_feat_pt_no_pad(feat, size = config_DK_AST.win_size, step = config_DK_AST.step_size,max_length = None )
                test_feat_list = list(test_feat_list)
                return test_feat_list,999
                
                
        

    
    
    
    
    ### Function to process the wav file #########
    @staticmethod
    def process_wav (filename,label_duration,label,pad = False,filter_signal = call_filter ):
        print("****** ENTERING PROCESS_WAV for file = " + str(filename) + "*****")
        #length = librosa.get_duration(filename = filename)
        bugs = 0    
        #to resamp to 16Khz
        # The below condition check any mismatch in labels
        #print("length from librosa = " + str(length) )
        #  assert math.isclose(length,label_duration, rel_tol=0.01), "File: %s label duration (%.4f) does not match audio length (%.4f)" % (row['path'], label_duration, length)
        signal_8k, rate = torchaudio.load(filename , normalize = True)
        #print("shape of signal_8k = " +str(signal_8k.shape))
        # resampling to 16 K as needed by AST
        transform = T.Resample(orig_freq = 8000,new_freq = 16000)
        signal_file = transform(signal_8k)
        #print("Post Resampling signal_16k = " +str(signal_file.shape))
        # if the incoming training file is padded or a mosquito - transform it with a mosq sound
        
        if (label == 1) :
            print("About to apply signal transform...")
            transformed_audio = compose_transform_aug_signal(signal_file)
            #print("Filename->" +str(filename) + "shape of transformed audio = " + str(transformed_audio.shape))
        else:
            print("About to apply background transform...")
            transformed_audio = compose_transform_aug_noise(signal_file)
        
        if pad == True :
            # this indicates that the function is being called from padding and the file has to be appended
                print("****** EXITING PROCESS_WAV (pad == TRUE)*****")
                return transformed_audio , label , None
        else:#when pad == false
            try:
                mel_spectrogram = T.MelSpectrogram(sample_rate=16000, n_fft=config_DK_AST.NFFT,win_length=config_DK_AST.win_size, 
                                                   hop_length=config_DK_AST.n_hop, center=True,
                                                   pad_mode="reflect", power=2.0, norm='slaney', onesided=True, n_mels=config_DK_AST.n_feat,
                                                   mel_scale="htk")
                melspec = mel_spectrogram(transformed_audio)
                db_obj= torchaudio.transforms.AmplitudeToDB(stype= 'power')
                feat = db_obj(melspec)
                #print("After melspectrogram . Shape of feat ->" + str(melspec.shape))
                if feat.shape[2] < config_DK_AST.win_size:
                    #print("Post Power_db. Shape = " + str(feat.shape))
                    print("RAISING VALUE ERROR..Power_db issue")
                    raise ValueError
                else:
                    feat_aug = AudioUtil.spec_augment(feat, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1)
                    #print("post spec augment . Shape = " + str(feat_aug.shape))
                    bugs = None
                    print("****** EXITING PROCESS_WAV*****")
                    return feat_aug ,label,bugs
                        
                        
            except ValueError:
                print("FILE SIZE TOOO SMALLLL ..error while processing wav..skipping..")
                bugs = filename
                feat_aug = -1
                y_temp = "FAIL"
                print("****** EXITING PROCESS_WAV*****")
                return feat_aug, y_temp,bugs
            
  
            
            
        
            
        
                    
                

## Pre Processing functions

## Modeling

In [9]:
class SoundDS(Dataset):
    
    def __init__(self,df,max_dim = None ,sr = config_DK_AST.rate ):
        self.df = df
        self.sr = sr
        self.channel = 1
        #self.feat_list = len(final_feat_list)
        print("len = " + str(len(self.df)))
               
  
  # Number of items in dataset
  # 
    def __len__(self):
        #all_spec_gram = AudioUtil.num_specgrams(self.df)
        #print("total number of specgram for the dataset is = " +str(all_spec_gram))
        
        return len(self.df)
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
    def __getitem__(self, idx , win_size = config_DK_AST.win_size , step_size = config_DK_AST.step_size , min_duration = config_DK_AST.min_duration,sr = config_DK_AST.sr):
        # Absolute file path of the audio file - concatenate the audio directory with
        # the relative path
        #get the file details from the idex passed from the train_loop
        print("inside get_item for train . Idx = " + str(idx))
        file_id = self.df.loc[idx,'id']
        label_df = self.df.loc[idx,'sound_type']
        
        # converting to numeric
        if label_df == 'mosquito':
            label  = 1
        else:
            label  = 0
        #filetag is the "name"   column in the df
        filetag = self.df.loc[idx,'name']
        _, file_format = os.path.splitext(filetag)
        filename_str = str(file_id) + file_format
        filename = os.path.join(data_dir, str(file_id) + file_format)
        label_duration = self.df.loc[idx,'length']
        print("^^^^^^^^ Start processing filename = " + str(filename_str) + "^^^^^^^^")
        #length = librosa.get_duration(filename = filename)
        #print("length from librosa = " + str(length) )
        return (filename_str , label)
    
    
         

In [10]:
class SoundDS_test(Dataset):
    
    def __init__(self,df ,sr = config_DK_AST.rate ):
        self.df = df
        self.sr = sr
        self.channel = 1
              
  # 
  # Number of items in dataset
  # 
    def __len__(self):
        #all_spec_gram = AudioUtil.num_specgrams(self.df)
        #print("total number of specgram for the dataset is = " +str(all_spec_gram))
        return len(self.df)
    
  # ----------------------------
  # Get i'th item in dataset
  # ----------------------------
    def __getitem__(self, idx , win_size = config_DK_AST.win_size , step_size = config_DK_AST.step_size , min_duration = config_DK_AST.min_duration,sr = config_DK_AST.sr):
        # Absolute file path of the audio file - concatenate the audio directory with
        # the relative path
        print("inside get_item for val/test. Idx = " +str(idx))
        file_id = self.df.loc[idx,'id']
        label_df = self.df.loc[idx,'sound_type']
        # converting to numeric
        if label_df == 'mosquito':
            label  = 1
        else:
            label  = 0
           
        filetag = self.df.loc[idx,'name']
        _, file_format = os.path.splitext(filetag)
        filename_str = str(file_id) + file_format
        filename = os.path.join(data_dir, str(file_id) + file_format)
        label_duration = self.df.loc[idx,'length']
        print("Test File :- filename = " + str(filename))
        return(filename_str,label)
        
           


In [11]:
def train_model_ast(train_dl,val_dl = None, model = ASTModel()):
    # X-train is a list of tensor and y_train is a list
    # we create a tuple of X-train and labels . This will help us to create a validation dataset.
              
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'Training on {device}')

    if torch.cuda.device_count() > 1:
        print("Using data parallel")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))

    
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimiser = optim.Adam(model.parameters(), lr=config_pytorch.lr)
    #print("Optim Device= " +str(optimiser.device))
    all_train_loss = []
    all_train_acc = []
    all_val_loss = []
    all_val_acc = []
    best_val_loss = np.inf
    best_val_acc = -np.inf
    best_train_acc = -np.inf
    best_epoch = -1
    checkpoint_name = None
    overrun_counter = 0
    for e in range(config_pytorch.epochs):
        start_epoch_time = time.time()
        print("epoch = " +str(e))
        train_loss = 0.0
        model.to(device).train()
        all_y = []
        all_y_pred = []
        total_sgrams = 0
        #our data is a list of dim = max_dims
        #dl_length  = int(len(train_dl))
        #loader_bat_size = min(bat_size,dl_length)
        for idx,train_bat in enumerate(train_dl):
            start_time = time.time()
            print("TRain_dl length is " + str(len(train_dl)))
            print("idx in train_ast = " +str(idx))
            file_bat = train_bat[0]
            label_bat = train_bat[1]
            print(" file bat = " + str(file_bat))
            print(" label bat = " + str(label_bat))
            for i in range(len(file_bat)):
                filename = file_bat[i]
                label = label_bat[i]
                print("Processing the file in train_ast->" + str(filename))
                print("filename = " +str(filename))
                print("label = " +str(label))
                #print("batch number ->" + str(i) + "of.. " + str(loader_bat_size))
                spec_list,status = AudioUtil.get_train_feat_list(filename,label)
                print("(inside train_ast) = len of of spec_list = " + str(len(spec_list)))
                if status == 999:
                    print("inside y = 999")
                    print("Failure occured for file and skipping this rec")
                    all_y.append(status)
                    continue
                
                total_sgrams+=int(len(spec_list))
                for j in range(len(spec_list)):
                    #print("index of spec_list = " + str(i))
                    y = torch.tensor(label).reshape(-1,1).to(device).float()
                    #print("%%%%%% INSIDE TRAIN LOOP(when there is no 999) TO GET A LOSS%%%%%%%%")
                    #print("index of spec_list = " + str(i))
                    x_temp,_ = spec_list[j]
                    #print("x_temp SHAPE = " + str(x_temp[0].shape))
                    x_temp_new = x_temp[0]
                    x_temp_new = x_temp_new.unsqueeze(dim = 0)
                    #print("shpe of x_temp_new = " + str(x_temp_new.shape))
                    #print("y SHAPE = " + str(y.shape))
                    # a zero tensor indicates a padding
                    # this is recommended in paper.
                    #print(x_reshaped.shape)
                    x_temp_new.cuda()
                    #print("x_temp_new device = " +str(x_reshaped.device))
                    y.cuda()
                    #print("Y device = " +str(y.device))
                    y_pred = model(x_temp_new.to(device).detach()).float()
                    #print("Y Pred Device = " +str(y_pred.device))
                    #print("Y  shape = " +str(y.shape))
                    #print("i = " + str(i))
                    #print("@@@@Y_pred = >" + str(y_pred))
                    #print("sigmoid of Y_pred = " +str(m(y_pred)))
                    loss = criterion(y_pred, y)
                    #print("loss  = " +str(loss.device))
                    optimiser.zero_grad()
                    loss.backward()
                    optimiser.step()
                    #print("Current train loss before adding the loss = " + str(train_loss))
                    train_loss += loss.detach().item()
                    #print("train loss - >" + str(loss.item()))
                    #print("train loss/len(train_loader)-> " + str(train_loss/len(train_loader)))
                    all_y.append(y.cpu().detach())
                    all_y_pred.append(y_pred.cpu().detach())
                    # if bat_size%100 == 0 :
                     #print("Inside Epoch " + str(e) + " & inside batch " + str(idx) + "specgram " + str(i) + " of " + str(len(tup_list))  )
                    del x_temp_new
                    del y
                    del y_pred

                    #print("Finished training a  batch")
                    finish_time = time.time()
                    duration = (finish_time - start_time )/60
                    if duration//120 > 1 :
                        print("Duration of training this file = " + str(duration)) 
                    #total_sgrams = len(train_dl) + item_in_bat
                    #print("total_sgrams = " + str(total_sgrams))
                    all_train_loss.append(train_loss/total_sgrams)    
                    #removing all instances of 999

        
        all_y_999_removed = all_y
        while 999 in all_y_999_removed: all_y_999_removed.remove(999)
        all_y_999_removed = torch.cat(all_y_999_removed)
        all_y_pred = torch.cat(all_y_pred)
        train_acc = accuracy_score(all_y_999_removed.numpy(), (all_y_pred.numpy() > 0.5).astype(float))
        all_train_acc.append(train_acc)
        
        # Can add more conditions to support loss instead of accuracy. Use *-1 for loss inequality instead of acc
        if val_dl is not None:
            val_loss, val_acc,total_val_size,invalid_file_flag = test_model(model, val_dl, criterion, 0.5, device=device)
            #when the val_data returns np.inf as loss and accuracy
            if invalid_file_flag == True:
                acc_metric = train_acc
                best_acc_metric = best_train_acc
            else:
                all_val_loss.append(val_loss)
                all_val_acc.append(val_acc)
                acc_metric = val_acc
                best_acc_metric = best_val_acc
        else:
            acc_metric = train_acc
            best_acc_metric = best_train_acc
        end_epoch_time = time.time()
        duration_epoch = (end_epoch_time -start_epoch_time )/60
        print("duration to train epoch " + str(e) + "= " + str(duration_epoch))
        if acc_metric > best_acc_metric:
            checkpoint_name = f'model_e{e}_{datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}.pth'
            torch.save(model.state_dict(), os.path.join(config_DK_AST.model_dir, 'pytorch', checkpoint_name))
            print('Saving model to:', os.path.join(config_DK_AST.model_dir, 'pytorch', checkpoint_name)) 
            best_epoch = e
            best_train_acc = train_acc
            best_train_loss = train_loss
            best_val_acc = val_acc
            best_val_loss = val_loss
            overrun_counter = -1

        overrun_counter += 1
        if (val_dl is not None) and (invalid_file_flag == False):
            print('Epoch: %d, Train Loss: %.8f, Train Acc: %.8f, Val Loss: %.8f, Val Acc: %.8f, overrun_counter %i' % (e, train_loss/(total_sgrams), train_acc, val_loss/(total_val_size), val_acc,  overrun_counter))
        else:
            print('Epoch: %d, Train Loss: %.8f, Train Acc: %.8f, overrun_counter %i' % (e, train_loss/(total_sgrams), train_acc, overrun_counter))
        if overrun_counter > config_pytorch.max_overrun:
            break
    return model




        

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=84


In [12]:
def evaluate_model(model, test_df):
    
    test_obj = SoundDS_test(test_df)
    test_dl = torch.utils.data.DataLoader(test_df, batch_size=16, shuffle=False)
    n_samples = len(test_df)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'Evaluating on {device}')
    print("n_samples = " +str(n_samples))

    #x_test,label = torch.tensor(X_test).float()
                
    y_preds_all = np.zeros([n_samples, len(y_test), 2])
    #print("Shape of y_preds_all = " + str(y_preds_all.shape))
    model.eval() # Important to not leak info from batch norm layers and cause other issues

    for n in range(n_samples):
        all_y_pred = []
        all_y = []
        
        for idx,(x, labels ) in enumerate(test_dl):
            # The model needs input in the form of(batchsize, time, freq . Reshaping the features below)           
            print("bat_idx = " + str(idx))
            feat_temp , labels  =  x, labels
            #print("shape of feat in the batch = " + str(feat_temp.shape))
            #print("Calling reshape ....")
            feat_list =  reshape_feat_pt_temp(feat_temp, labels, size = config_DK_AST.win_size, step = config_DK_AST.step_size)
            #This is of the form [1,var,80,128]
            #print("Batch#->" + str(idx) + "will have " + str(len(feat_list)) + " elements")          
            one_image_loss = 0
            for i in range(0,len(feat_list)):
                x_temp,y_elem = feat_list[i]
                bat_size = x_temp.shape[0]
                time_dim = x_temp.shape[1]
                freq_dim = x_temp.shape[2]
                                
                #print("X-temp Shape = " +str(x_temp.shape))
                #x_reshaped = x_temp.reshape(bat_size,time_dim,freq_dim)
                
            # this is recommended in paper.
            #x_reshaped = ((x_reshaped - train_data_mean)/(train_data_sd)).cuda()
                y = y_elem.reshape(-1,1).to(device).float()
                #print("Lables = " +str(y))
                #print("Type of Y = " + str(type(y)))
                #print("Unsqueeze Y = " + str(y))
                #print(x_reshaped.shape)
                optimiser.zero_grad()
                x_reshaped.cuda()
                #print("X_REshaped device = " +str(x_reshaped.device))
                y.cuda()
            #print("Y device = " +str(y.device))
                y_pred = model(x_reshaped.to(device).detach()).float()
                all_y.append(y.cpu().detach())
                all_y_pred.append(y_pred.cpu().detach())
                del x
                del y
                del y_pred
                all_y_pred = torch.cat(all_y_pred)
                all_y = torch.cat(all_y)

        y_preds_all[n,:,1] = np.array(all_y_pred)
        y_preds_all[n,:,0] = 1-np.array(all_y_pred) # Check ordering of classes (yes/no)
        test_acc = accuracy_score(all_y.numpy(), (all_y_pred.numpy() > 0.5).astype(float))
        # print(test_acc)
    return y_preds_all



In [13]:
def evaluate_model_aggregated(model, X_test, y_test, n_samples):
    ''' Generate predictions for VGGish features (Feat. A) rescaled to time window of 1.92 second features (Feat. B)'''
    preds_aggregated_by_mean = []
    y_aggregated_prediction_by_mean = []
    y_target_aggregated = []
    
    for idx, recording in enumerate(X_test):
        n_target_windows = len(recording)//2  # Calculate expected length: discard edge
        y_target = np.repeat(y_test[idx],n_target_windows) # Create y array of correct length
        preds = evaluate_model(model, recording, np.repeat(y_test[idx],len(recording)),n_samples) # Sample BNN
#         preds = np.mean(preds, axis=0) # Average across BNN samples
#         print(np.shape(preds))
        preds = preds[:,:n_target_windows*2,:] # Discard edge case
#         print(np.shape(preds))
#         print('reshaping')
        preds = np.mean(preds.reshape(len(preds),-1,2,2), axis=2) # Average every 2 elements, keep samples in first dim
#         print(np.shape(preds))
        preds_y = np.argmax(preds)  # Append argmax prediction (label output)
        y_aggregated_prediction_by_mean.append(preds_y)
        preds_aggregated_by_mean.append(preds)  # Append prob (or log-prob/other space)
        y_target_aggregated.append(y_target)  # Append y_target
#     return preds_aggregated_by_mean, y_aggregated_prediction_by_mean, y_target_aggregated
    return np.hstack(preds_aggregated_by_mean), np.concatenate(y_target_aggregated)


In [14]:
def reshape_feat_pt_no_pad(feats,label,size = config_DK_AST.win_size, step = config_DK_AST.step_size):
    #tup_list will hold the values in the form of an image(x,label)
    print("&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&")
    #print("initial shape of feat = " + str(feats.shape))
    feat_list = []
    feat_first_tpose = torch.transpose(feats,1,2)
    print("shape after first transpose = " + str(feat_first_tpose.shape))
    feat_new = feat_first_tpose.unfold(1,size  = config_DK_AST.win_size, step = config_DK_AST.step_size)
    #print("shape after unfolding = " + str(feat_new.shape))
    feat_squee = feat_new.squeeze()
    #print("shape post squeeze = " + str(feat_squee.shape))
    len_squeeze = list(feat_squee.shape)
    if len(len_squeeze) == 2:
        if feat_squee.shape[1] < config_DK_AST.win_size :
            # 999 indicated error condition
            #print("inside feat_squee.shape[1] < config_DK_AST.win_size")
            feature_zero = list(torch.zeros(1,config_DK_AST.win_size,config_DK_AST.n_feat))
            print("returning zeros and 999")
            tup = (feature_zero,999)
            feat_list.append(tup)
            print("returning a feat_list with elements = " + str(len(feat_list)))
            return feat_list
            
        else:
            #print("already in desired shape")
            t = torch.transpose(feat_squee,1,0)
            t = torch.unsqueeze(t,dim = 0)            
            #print("shape of feat = " + str(t.shape))
            feature = list(t)
            tup = (feature,label)
            print("returning a feat_list with elements = " + str(len(feat_list)))
            feat_list.append(tup)
            return feat_list
              
    else:
        num_chunk = feat_squee.shape[0]
        #print("Num chunks to be generated = " + str(num_chunk))
        chunk_list = torch.chunk(feat_squee , num_chunk , 0)
        #print("One sound file being split into  " + str(len(chunk_list)))
        #print("LEngth of list of tensors = " + str(len(chunk_list)))
        
        for s_gram in (chunk_list):
            #print("sgram shape =" + str(torch.transpose(s_gram,2,1).shape))
            #print("ctr =" +str(ctr))
            feature = torch.transpose(s_gram,2,1)
            #print("before appending feat_list = " + str(len(feat_list)))
            tup = (list(feature),label)
            feat_list.append(tup)
            #print("POST appending feat_list = " + str(len(feat_list)))
            
        print("returning a feat_list with elements = " + str(len(feat_list)))
        return feat_list
    
    
    

In [15]:
def test_model(model, val_dl, criterion, class_threshold=0.5, device=None):
        
    test_loss = 0.0
    model.eval()
    all_y = []
    all_y_pred = []
    with torch.no_grad():
        total_sgram_val = 0
        print("^^^^^ INSIDE VAL DATA^^^^^^^^^")
        for idx,val_batch in enumerate(val_dl):
            #print("filename without de-listing " +str(filename))           
            print("val_dl length is = " +str(len(val_dl)))
            print("idx in val_dl = " +str(idx))
            file_bat = val_batch[0]
            label_bat = val_batch[1]
            print(" file bat = " + str(file_bat))
            print(" label bat = " + str(label_bat))
            for i in range(len(file_bat)):
                filename = file_bat[i]
                label = label_bat[i]
                print("Processing the file in train_ast->" + str(filename))
                print("filename = " +str(filename))
                print("label = " +str(label))
                start_time = time.time()
                total_sgram_list_val,status = AudioUtil.get_test_fest_list(filename,label)
            #print("Total num os spec grams returned = " + str(total_sgram_val))
                if status == 999 :
                    print("999 encountered .Error file..skipping")
                    all_y.append(status)
                    print("Length of all_y = " +str(len(all_y)))
                    continue
                
                total_sgram_val+=(len(total_sgram_list_val))
                #print("Num items in this list = " + str(len(x_el_val)))
                for i in range(len(total_sgram_list_val)):
                    y = label.reshape(-1,1).to(device).float()
                    x_temp_val,_ = total_sgram_list_val[i]
                    x_temp_new = x_temp_val[0].unsqueeze(dim=0)
                    #print("x_temp SHAPE = " + str(x_temp_val.shape))
                    #print("y SHAPE = " + str(y.shape))
                    # a zero tensor indicates a padding
                    #bat_size = x_temp.shape[1]
                    #time_dim = x_temp.shape[2]
                    #freq_dim = x_temp.shape[3]
                    #x_reshaped = x_temp.reshape(bat_size,time_dim,freq_dim)
                    #print("X_REshaped shape = " +str(x_reshaped.shape))
                    x_temp_new.to(device)
                    #print("X_REshaped device = " +str(x_reshaped.device))
                    #y.to(device)
                    #print("Y device = " +str(y.device))
                    y_pred = model(x_temp_new.to(device).detach()).float()
                    #print("Y Pred Device = " +str(y_pred.device))
                    #print("Y Pred shape = " +str(y_pred.shape))
                    #print("Y  shape = " +str(y.shape))
                    #print("@@@@Y_pred in val= >" + str(y_pred))
                    loss = criterion(y_pred, y)
                    test_loss += loss.detach().item()
                    #print("loss  = " +str(loss.device))
                    #print("train loss/len(train_loader)-> " + str(train_loss/len(train_loader)))
                    all_y.append(y.cpu().detach())
                    all_y_pred.append(y_pred.cpu().detach())
                    #if bat_size%100 == 0 :
                    #    print("Inside Epoch " + str(e) + " & inside batch " + str(idx) + "specgram " + str(i) + " of " + str(len(item_in_bat))  )
                    del x_temp_val
                    del x_temp_new
                    del y
                    del y_pred


                #removing instances of999
            
        all_y_999_removed = all_y
        while 999 in all_y_999_removed: all_y_999_removed.remove(999)
        #print("all_y_999_removed = " + str(all_y_999_removed))
        #print("all_y = " + str(all_y))             
        #all_y = torch.cat(all_y)
        #all_y_999_removed = torch.cat(all_y_999_removed)
        #all_y_pred = torch.cat(all_y_pred)
            
        if(len(all_y_999_removed) == 0):
            test_loss = np.inf
            test_acc = -np.inf
            total_val_size = 1
            invalid_file_flag = True
            return test_loss, test_acc,total_val_size,invalid_file_flag
                
                
                
        else:
            test_loss = test_loss/(total_sgram_val)
            all_y_999_removed = torch.cat(all_y_999_removed)
            all_y_pred = torch.cat(all_y_pred)
            #test_acc = accuracy_score(all_y_999_removed.numpy(), (all_y_pred.numpy() > class_threshold).astype(float))
            #print("all_y_999_removed = " + str(np.array(all_y_999_removed)))
            #print("all_y_pred = " + str(np.array(all_y_pred)))
            test_acc = accuracy_score(all_y_999_removed.numpy(), all_y_pred.numpy() > class_threshold).astype(float)
            #               test_acc = accuracy_score(np.array(all_y_999_removed), np.array(all_y_pred)).astype(float)
            
            total_val_size = total_sgram_val
            invalid_file_flag = False
            #print("total_val_size= " + str(total_val_size))


            return test_loss, test_acc,total_val_size,invalid_file_flag 

## Data Load and Training

In [16]:
library = 'PyTorch'

if library == 'PyTorch':
    from PyTorch.runTorch_AST_DK import (ASTModel)

In [17]:
import pandas as pd
df = pd.read_csv(config_DK_AST.data_df_dk_1)

# To be kept: please do not edit the test set: these paths select test set A, test set B as described in the paper
idx_test_A = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'field')
idx_test_B = np.logical_and(df['country'] == 'UK', df['location_type'] == 'culture')
idx_train = np.logical_not(np.logical_or(idx_test_A, idx_test_B))
df_test_A = df[idx_test_A]
df_test_B = df[idx_test_B]


df_train = df[idx_train]

# Modify by addition or sub-sampling of df_train here


# Assertion to check that train does NOT appear in test:
assert len(np.where(pd.concat([df_train,df_test_A,
                               df_test_B]).duplicated())[0]) == 0, 'Train dataframe contains overlap with Test A, Test B'





In [18]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df_train, test_size=0.1,shuffle = True)
print("len(train) = "  +str(len(train)))
print("len(val) = "  +str(len(val)))

train = train.reset_index(drop=True)
val =  val.reset_index(drop=True)

len(train) = 18
len(val) = 3


In [19]:
val

,id,length,name,sample_rate,record_datetime,sound_type,species,gender,fed,plurality,age,method,mic_type,device_type,country,district,province,place,location_type
0,74,0.127687,CDC_Ae-aegypti_labelled_800.wav,8000,8/9/2016 8:00,mosquito,ae aegypti,NaN,NaN,Single,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture
1,219424,5.376000,r2015-12-17_18.30.08.828.wav_u2016-04-08_14.22...,8000,17-12-15 18:30,background,NaN,NaN,NaN,NaN,NaN,NaN,phone,Alcatel 4015X,Kenya,NaN,Nairobi,USAMRU-K,culture
2,63,0.080395,CDC_Ae-aegypti_labelled_800.wav,8000,8/9/2016 8:00,mosquito,ae aegypti,NaN,NaN,Single,NaN,NaN,phone,Alcatel 4009X,USA,Georgia,Atlanta,"CDC insect cultures, Atlanta",culture


In [20]:

train_obj = SoundDS(train )
val_obj = SoundDS_test(val)


# Create training and validation data loaders
#train_dl = torch.utils.data.DataLoader(train_obj, batch_size=16, shuffle=True,num_workers = 2 , collate_fn = collate_data,pin_memory = True)
train_dl = torch.utils.data.DataLoader(train_obj, batch_size= loader_bat_size, shuffle=True,pin_memory = True)
val_dl = torch.utils.data.DataLoader(val_obj, batch_size= loader_bat_size, shuffle=False,pin_memory = True)

# train_dl = torch.utils.data.DataLoader(train_obj, batch_size= 1, shuffle=True, collate_fn = collate_data,pin_memory = True)
# val_dl = torch.utils.data.DataLoader(val_obj, batch_size=1, shuffle=False,collate_fn = collate_data,pin_memory = True)

# print("Len train DL = " + str(len(train_dl)))
# print("Len VAL DL = " + str(len(val_dl)))




len = 18


In [21]:
dataiter = iter(train_dl)
data = dataiter.next()
a,b = data
print(a)
print(b)
# data_x,data_y = data
# print((data_x))
# print((data_y[0]))

# dl_length  = len(train_dl)
# loader_bat_size = min(4,dl_length)

inside get_item for train . Idx = 6
^^^^^^^^ Start processing filename = 219432.wav^^^^^^^^
inside get_item for train . Idx = 11
^^^^^^^^ Start processing filename = 54.wav^^^^^^^^
inside get_item for train . Idx = 14
^^^^^^^^ Start processing filename = 86.wav^^^^^^^^
inside get_item for train . Idx = 1
^^^^^^^^ Start processing filename = 72.wav^^^^^^^^
['219432.wav', '54.wav', '86.wav', '72.wav']
tensor([0, 1, 1, 1])


### Get the max dims for the val_data

In [22]:
#train

In [23]:
#val

In [24]:
!nvidia-smi

Sun Mar  6 06:40:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   31C    P0    40W / 300W |   1373MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model = train_model_ast(train_dl , val_dl )

Training on cuda:0
epoch = 0
inside get_item for train . Idx = 8
^^^^^^^^ Start processing filename = 73.wav^^^^^^^^
inside get_item for train . Idx = 13
^^^^^^^^ Start processing filename = 218851.wav^^^^^^^^
inside get_item for train . Idx = 11
^^^^^^^^ Start processing filename = 54.wav^^^^^^^^
inside get_item for train . Idx = 10
^^^^^^^^ Start processing filename = 218964.wav^^^^^^^^
TRain_dl length is 5
idx in train_ast = 0
 file bat = ['73.wav', '218851.wav', '54.wav', '218964.wav']
 label bat = tensor([1, 1, 1, 0])
Processing the file in train_ast->73.wav
filename = 73.wav
label = tensor(1)
inside get feat list , processing ->../data/audio/73.wav
inside shorter files
****** ENTERING PROCESS_WAV for file = ../data/audio/73.wav*****
About to apply signal transform...
****** EXITING PROCESS_WAV (pad == TRUE)*****
......APplying spec augment.....
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 83, 128])
returning a feat_list with

/tmp/ipykernel_2104/2368998103.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(label).reshape(-1,1).to(device).float()


Processing the file in train_ast->218851.wav
filename = 218851.wav
label = tensor(1)
inside get feat list , processing ->../data/audio/218851.wav
****** ENTERING PROCESS_WAV for file = ../data/audio/218851.wav*****
About to apply signal transform...
......APplying spec augment.....
****** EXITING PROCESS_WAV*****
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 101, 128])
returning a feat_list with elements = 5
(inside train_ast) = len of of spec_list = 5
Processing the file in train_ast->54.wav
filename = 54.wav
label = tensor(1)
inside get feat list , processing ->../data/audio/54.wav
inside shorter files
****** ENTERING PROCESS_WAV for file = ../data/audio/54.wav*****
About to apply signal transform...
****** EXITING PROCESS_WAV (pad == TRUE)*****
......APplying spec augment.....
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 81, 128])
returning a feat_list with elements = 0
(in

/tmp/ipykernel_2104/2368998103.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(label).reshape(-1,1).to(device).float()


......APplying spec augment.....
****** EXITING PROCESS_WAV*****
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 83, 128])
returning a feat_list with elements = 0
(inside train_ast) = len of of spec_list = 1
Processing the file in train_ast->219427.wav
filename = 219427.wav
label = tensor(0)
inside get feat list , processing ->../data/audio/219427.wav
****** ENTERING PROCESS_WAV for file = ../data/audio/219427.wav*****
About to apply background transform...
......APplying spec augment.....
****** EXITING PROCESS_WAV*****
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 1457, 128])
returning a feat_list with elements = 276
(inside train_ast) = len of of spec_list = 276
Processing the file in train_ast->219430.wav
filename = 219430.wav
label = tensor(0)
inside get feat list , processing ->../data/audio/219430.wav
****** ENTERING PROCESS_WAV for file = ../data/audio/219430.wav*****
Abo

/tmp/ipykernel_2104/2368998103.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(label).reshape(-1,1).to(device).float()


Processing the file in train_ast->72.wav
filename = 72.wav
label = tensor(1)
inside get feat list , processing ->../data/audio/72.wav
inside shorter files
****** ENTERING PROCESS_WAV for file = ../data/audio/72.wav*****
About to apply signal transform...
****** EXITING PROCESS_WAV (pad == TRUE)*****
......APplying spec augment.....
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 86, 128])
returning a feat_list with elements = 2
(inside train_ast) = len of of spec_list = 2
Processing the file in train_ast->218964.wav
filename = 218964.wav
label = tensor(0)
inside get feat list , processing ->../data/audio/218964.wav
****** ENTERING PROCESS_WAV for file = ../data/audio/218964.wav*****
About to apply background transform...
......APplying spec augment.....
****** EXITING PROCESS_WAV*****
&&&&&&&&&&&&&&&& INSIDE RESHAPE &&&&&&&&&&&&&&&&&&&&&&&
shape after first transpose = torch.Size([1, 153, 128])
returning a feat_list with elements = 1

# Training

In [26]:
!pwd

/dli/task/Capstone-HumBug/notebooks


In [26]:
#x = reshape_feat_pt(torch.random(1,128,408))
librosa.get_duration(filename = '../data/audio/71.wav')

0.690375

In [ ]:

#model = train_model_ast(X_train, y_train, X_val, y_val, model=ASTModel())
#model = train_model_ast(train_data, val_data,model=ASTModel())

## Evaluation

In [ ]:
max_dim_testA = get_max_file_dims(df_test_A )
max_dim_testA = max_dim_testA + int(max_dim_testA*config_DK_AST.len_tol)


max_dim_testB = get_max_file_dims(df_test_B )
max_dim_testB = max_dim_testB + int(max_dim_testB*config_DK_AST.len_tol)


testA_obj = SoundDS_test(df_test_A,max_dim = max_dim_testA)
testB_obj = SoundDS_test(df_test_B , max_dim = max_dim_testA)

In [ ]:
def load_model(filepath, model=ASTModel()):
    # Instantiate model to inspect
    print("Filepath = " + str(filepath))
    print("model = " +str(model))
    device = torch.device('cuda:0' if torch.cuda.is_available() else torch.device("cpu"))
    print(f'Training on {device}')
        
    if torch.cuda.device_count() > 1:
        print("Using data parallel")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    model = model.to(device)
    # Load trained parameters from checkpoint (may need to download from S3 first)


    if torch.cuda.is_available():
        map_location=lambda storage, loc: storage.cuda()
    else:
        map_location='cpu'
        
    checkpoint = model.load_state_dict(torch.load(filepath))

    return model

In [ ]:
#mlp = MLP()
#mlp.load_state_dict(torch.load(save_path))


model_init=ASTModel()
model_init.eval()
path = '../outputs/models/pytorch/'
model_name = 'model_e2_2022_02_14_21_57_53.pth'


#checkpoint = model.load_state_dict(torch.load(filepath))
trained_model = load_model(path + model_name , model = model_init)


# Dummy data for evaluation

#creating random data to test 
#f1 = np.random.randn(128,1075)
#f2 = np.random.randn(128,242)
#f3 = np.random.randn(128,234)
#f4 = np.ranadom.randn(128,263)

# replicates get_feat
#feats = [f1,f2,f3,f4]
#labels = [0,1,0,1]



## Test A

In [ ]:
feat_type = 'FeatB'
if feat_type == 'FeatA':
    p, yt = evaluate_model_aggregated(trained_model, X_test_A, y_test_A, 1)  # Aggregate windows from feature list (0.96->1.92 s)
    PE, MI, log_prob = get_results(p, yt, filename = feat_type + '_' + model_name +'_Test_A')
elif feat_type == 'FeatB':
    y_preds_all = evaluate_model(trained_model, X_test_A, y_test_A, 1)  # Predict directly over feature windows (1.92 s)
    PE, MI, log_prob = get_results(y_preds_all, y_test_A, filename = feat_type + '_' + model_name +'_Test_A')

## TEST B

In [ ]:
feat_type = 'FeatB'
if feat_type == 'FeatA':
    
    p, yt = evaluate_model_aggregated(model, X_test_B, y_test_B, 1)  # Aggregate windows from feature list (0.96->1.92 s)
    PE, MI, log_prob = get_results(p, yt, filename = feat_type + '_' + model_name +'_Test_B')
elif feat_type == 'FeatB':
    y_preds_all = evaluate_model(trained_model, X_test_B, y_test_B, 1)  # Predict directly over feature windows (1.92 s)
    PE, MI, log_prob = get_results(y_preds_all, y_test_B, filename = feat_type + '_' + model_name +'_Test_B')